# Motivation

Hi everyone. I think that _every_ student should have some ability to write code. In our digital world, being fluent in some programming language is almost as important as your ability to read and write. Sure, you can survive _without_ it ... but your are so much more valuable as a scientist and employee _with_ it. Acorrding to some research, high-paying jobs with coding requirements pay about \\$22,000 more (on average) than jobs that don't require coding. Be aware that this is correlation, not causation. Is a chemist that knows how to code going to make an extra \\$22,000 per year compared to a chemist that doesn't know how to code? That seems unlikely. However, I am confident that, with all other variables being equal, a ~~chemist~~ scientist that can code will make more money and have more job opportunities than a ~~chemist~~ scientist that cannot code.

So, am I expecting this one excercise to make you into a marketable coder? Absolutely not. I know that coding is a skill and it takes time to develop that skill. But you need some place to start...

Here are the things that I want you to take home from this exercise:
* See the differences between real and complex Fourier transforms
* Gain a deeper understanding of quadrature detection
* Make connections between the theories and equations that we discuss in class and the real life phenomenon and observables that you see in lab
* See coding as a tool to help you simulate complex numerical problems
* See coding as an extension or alternative to Excel
* See how easy it is to produce consistent, publication-quality figures
* See that coding a simulation like this is a great way to understand all of the theory behind that simulation

To that end, I am introducing you the the Python programming language - arguably the hottest, most in-demand, and easiest language to learn. And I'm doing that with something called Jupyter. Jupyter is a web-based programming environment that was designed to: 
* run in a web browser
* combine code, documentation, comments, and graphics
* focus on the programming languages **JU**lia, **PYT**hon, and **R**
* emulate the look and feel of software like Mathematica

To give you some context, Instagram uses [Python](https://www.zdnet.com/article/programming-languages-how-instagrams-taming-a-multimillion-line-python-monster/) and Netflix uses [Jupyter](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233). Want to work for either of those companies? In general, scientists are good with data and statistics; these are exactly the job skills that hiring managers are looking for at companies like Netflix and Instagram. It's important to make note that I'm not talking about computer scientists or software engineers. I'm talking about data scientists - LITERALLY THE STUFF THAT WE DO EVERY DAY!

# Task 0

This is called a _markdown_ cell. It can contain text, links, graphics, equations, documentation, comments, etc. I'm going to use these cells to give you instructions.

Click on it. See that blue vertical bar that appeared in the left margin? That tells you that this cell is "active".

Double-click this cell. See how the text changed from nicely formatted text to ugly, unformatted text? This is analogous to looking at a web page versus looking at the HTML code that makes that web page. Press **SHIFT+ENTER** to execute this cell and go back to the formatted text.

The cell below this one is called a _code_ cell. It contains Python code and can contain a few commands or an entire program. Run that cell (remember, **SHIFT+ENTER**) and see what happens. After you do that, go back to that cell and change the code so that it prints **Hello, [YOUR NAME]**. Run it again. Did it work?



In [ ]:
print("Hello, world!")

# Scientific background

We have been talking about the **FID** (**F**ree **I**nduction **D**ecay). This is the _time-domain_ signal that we get straight out of the spectrometer. We can represent the FID mathematically as a damped cosine function. Technically, it should be a sum of damped cosine functions (one for each peak that we see in our spectrum) but we're trying to simplify things here. If you need an example, consider a sample that contains $H_2O$ as the analyte. There are two equivalent ${^1}H's$ so we see one peak in our spectrum. We can model the FID for this sample as:
$$f(x)=cos({\omega}2{\pi}t)e^{\frac{-t}{T_2}}$$

After we do the Fourier transform of our spectrum, we get a peak with a frequency of $\omega$. Right? Let's find out...

# Task 1

Run the cell below. This should produce two plots:
* a damped cosine with a frequency of _OMEGA_ (in this example, it's set to 20 Hz)
* the Fourier transform of that signal

In [ ]:
# import the libraries so we can do cool math stuff and make pretty plots
import numpy as np
import matplotlib.pyplot as plt

# define a standard look for all of the plots
%matplotlib inline
font = {'size'   : 20}
plt.rc('font', **font)

# define all of our constants for these simulations
NP = 16384             # number of data points; must be 2^n
OMEGA = 20.0           # the frequency of our peak
DECAY = 3.0            # this is R2 which equals 1/T2
DWELL = 1 / 8000.0     # calculate the dwell time based on the sweep width
AQ = NP * DWELL        # the acquisition time is just the number of data points times the delay between points
LLIM = 30.0            # left limit
RLIM = -30.0           # right limit - yes, I know negative numbers should be on the left, but NMR has weird conventions
BLIM = -1500.0         # bottom limit
TLIM = 1500.0          # top limit

# create the time points that we want to use for the sin and cos calculations; create NP data points between 0.00 - AQ seconds
x = np.linspace(0.00, AQ, NP)

# calculate the damped oscillation - we use 2*pi*omega to get units of Hertz instead of rad/s
y = np.cos(OMEGA * 2 * np.pi * x) * np.exp(-x * DECAY)

# do the Fourier transform and calculate the frequencies for the x-axis
ft_y = np.fft.fft(y)
freq = np.fft.fftfreq(NP) / DWELL

# make the stacked plots
fig, axs = plt.subplots(2, 1, figsize=(15,7.5))
axs[0].plot(x, y)
axs[0].set_xlim(0.00, AQ)
axs[0].set_xlabel('time')
axs[0].set_ylabel('intensity')
axs[0].set_ylim(-1.00, 1.00)
axs[0].grid(True)

axs[1].plot(freq, ft_y.real)
axs[1].set_xlim(LLIM, RLIM)
axs[1].set_xlabel('frequency')
axs[1].set_ylabel('intensity')
axs[1].set_ylim(BLIM, TLIM)
axs[1].grid(True)

fig.tight_layout()
plt.show()

# Reflection 1

Let's talk about what you see and what you expected to see. We'll play a game to see if we can figure out what happened.

# Task 2

Hmm. So a _real_ Fourier transform has ambiguity that you observe as peaks at $\pm\omega$. Let's see if we can fix that. We could cut the spectrum in half and discard the right half. Wouldn't that get rid of the false peak? Maybe there's a better way?

I've already given you a clue by emphasizing that we used a _real_ Fourier transform. What about a _complex_ Fourier transform? Where do we get the imaginary component to do a complex FT? First, we'll check the math to make sure it's feasible. If it works out, then we'll worry about how to make it happen physically.

Let's compare the real Fourier transforms of cos and sin and see if that helps us.

Run the code cell below

In [ ]:
yc = np.cos(OMEGA * 2 * np.pi * x) * np.exp(-x * DECAY)
ys = np.sin(OMEGA * 2 * np.pi * x) * np.exp(-x * DECAY)

ft_yc = np.fft.fft(yc)
ft_ys = np.fft.fft(ys)

fig, axs = plt.subplots(2, 1, figsize=(15,7.5))
axs[0].plot(freq, ft_yc.real)
axs[0].set_xlim(LLIM, RLIM)
axs[0].set_xlabel('frequency')
axs[0].set_ylabel('intensity')
axs[0].set_ylim(BLIM, TLIM)
axs[0].grid(True)

axs[1].plot(freq, ft_ys.imag)
axs[1].set_xlim(LLIM, RLIM)
axs[1].set_xlabel('frequency')
axs[1].set_ylabel('intensity')
axs[1].set_ylim(BLIM, TLIM)
axs[1].grid(True)

fig.tight_layout()
plt.show()

# Reflection 2

Interesting. Remember that we defined _OMEGA_ to be 20 Hz at the very beginning of this exercise. The true signals at $\omega$ are both positive but the false signals at $-\omega$ have opposite signs. What if we take the difference of these two signals?!? Mathematically, that's very easy to do: $cos({\omega}t)-sin({\omega}t)$. Done.

But how do we generate those two signals in the instrument so that we can treat one as the real component and one as the imaginary component? If you were going to explain the difference between sin and cos to someone, what would you tell them?

Let's go back to my game ...

# Task 3

Let's bring all of this together and see what we end up with.

* We know that the Fourier transform of a real signal is no good.
* We know that we have to generate a complex signal
* We know that it's as simple as putting two detectors around the probe: one at $0^{\circ}$ and one at $90^{\circ}$ - one for the sin component and the other for the cos component

Run the code cell below

In [ ]:
ycomp = yc + (ys * 1j)
ft_yc = np.fft.fft(ycomp)

fig, axs = plt.subplots(2, 1, figsize=(15,7.5))
axs[0].plot(x, yc, x, ys)
axs[0].set_xlim(0.00, AQ)
axs[0].set_xlabel('time')
axs[0].set_ylabel('intensity')
axs[0].set_ylim(-1.00, 1.00)
axs[0].grid(True)

axs[1].plot(freq, ft_yc.real)
axs[1].set_xlim(LLIM, RLIM)
axs[1].set_xlabel('frequency')
axs[1].set_ylabel('intensity')
axs[1].set_ylim(BLIM, 2 * TLIM)
axs[1].grid(True)

fig.tight_layout()
plt.show()

# Reflection 3

Is this better? Is this what you expected for a sample that contains only $H_2O$ as the analyte?

Would you be mad if I told you that there was no "second detector"? :-)